## Imports

In [1]:
import findspark

findspark.init('C:/spark')

In [2]:
from pyspark.sql import SparkSession

In [3]:
from pyspark.ml.regression import LinearRegression

## Create a Session

In [4]:
spark = SparkSession.builder.appName('linreg').getOrCreate()

## Read Data

In [5]:
data = spark.read.csv('../../data/Ecommerce_Customers.csv', inferSchema=True, header=True)

In [6]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [7]:
data.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)]

## MLlib Imports

In [9]:
from pyspark.ml.feature import VectorAssembler

## Data Transformation

In [10]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [11]:
assembler = VectorAssembler(inputCols=['Avg Session Length',
                                       'Time on App',
                                       'Time on Website',
                                       'Length of Membership'],
                            outputCol='features')

In [12]:
output = assembler.transform(data)

In [13]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [14]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [15]:
final_data = output.select('features', 'Yearly Amount Spent')
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

## Train Test Split

In [16]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [17]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                349|
|   mean| 499.16120893201213|
| stddev|  74.50331548558978|
|    min|   266.086340948469|
|    max|  725.5848140556806|
+-------+-------------------+



In [18]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                151|
|   mean| 499.66726630478956|
| stddev|  89.71958319787971|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



## Model

In [19]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [20]:
lr_model = lr.fit(train_data)

In [21]:
test_results = lr_model.evaluate(test_data)

In [22]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|-12.799087018919977|
|-6.2562833561149205|
|-0.8370751984483036|
|  9.559892250207781|
| -13.80857233696969|
|  3.122805688483936|
| -4.816330037690477|
| -6.067278717885756|
|  -4.89553029143832|
|  -6.18615295103325|
| -18.49473409438383|
|-2.7006018616617666|
|  16.29462485404298|
| -5.362478969557571|
|-27.111068807548577|
| -5.269404550811089|
|  -4.20001684363524|
| -2.835943740179289|
| -2.630914381605521|
|-11.098379655494568|
+-------------------+
only showing top 20 rows



C:\spark\python\pyspark\sql\context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [23]:
test_results.rootMeanSquaredError

9.412229446445174

In [24]:
test_results.r2

0.9889211111623426

## Unlabeled

In [25]:
unlabeled_data = test_data.select('features')

In [26]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.3931845423455...|
|[30.4925366965402...|
|[30.5743636841713...|
|[30.7377203726281...|
|[31.0662181616375...|
|[31.3584771924370...|
|[31.4252268808548...|
|[31.5147378578019...|
|[31.5171218025062...|
|[31.5257524169682...|
|[31.5702008293202...|
|[31.5761319713222...|
|[31.6098395733896...|
|[31.6253601348306...|
|[31.6739155032749...|
|[31.7656188210424...|
|[31.8124825597242...|
|[31.8186165667690...|
|[31.8627411090001...|
|[31.8648325480987...|
+--------------------+
only showing top 20 rows



In [27]:
predictions = lr_model.transform(unlabeled_data)

In [28]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.3931845423455...| 332.7279568221136|
|[30.4925366965402...|288.72752907602944|
|[30.5743636841713...|442.90148895651396|
|[30.7377203726281...| 452.2208499460221|
|[31.0662181616375...|462.74186554464404|
|[31.3584771924370...| 492.0531447609915|
|[31.4252268808548...| 535.5830486924524|
|[31.5147378578019...|495.87976671434717|
|[31.5171218025062...|280.81395094182403|
|[31.5257524169682...|450.15177976091513|
|[31.5702008293202...| 564.4402262357887|
|[31.5761319713222...| 543.9271858509901|
|[31.6098395733896...| 428.2509247970652|
|[31.6253601348306...|381.69937972648177|
|[31.6739155032749...| 502.8361367174298|
|[31.7656188210424...| 501.8234861864182|
|[31.8124825597242...|397.01036182743246|
|[31.8186165667690...|449.25461711031494|
|[31.8627411090001...| 558.9290555556522|
|[31.8648325480987...|450.98966013230825|
+--------------------+------------